In [36]:
#protein solubility prediction
#interactive inline plot
%matplotlib notebook 

#Import all the packages
import pandas as pd
import numpy as np

from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_validate
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.neural_network import MLPRegressor
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import average_precision_score
from sklearn.metrics import make_scorer
from sklearn import svm

In [22]:
#first defind the headers of the columns (the different proteins) and then load data into dataframe
headers = ['A', 'R', 'N', 'D', 'C', 'E', 'Q', 'G', 'H', 'I', 'L', 'K', 'M', 'F', 'P', 'S', 'T', 'W', 'Y', 'V','solubility']
#original data
df_Original = pd.read_csv("clean14w.csv",index_col=0,names=headers,header=0)
df_Original.dropna(axis=0,inplace=True)
#gan supplemented data
df_WithGan = pd.read_csv("totalCGAN7.csv",index_col=0,names=headers,header=0)
df_WithGan.dropna(axis=0,inplace=True)

In [95]:
def Neural_Network_Prediciton(df):
    #set the X(proteins) and y(soubility)
    X = df.iloc[:,0:-1];
    y = df.solubility;

    X = X.as_matrix().astype(np.float)
    y = y.as_matrix().astype(np.float)

    #set 10 fold validation
    kf = KFold(n_splits=10)

    def mc_reg(y_true, y_pred):
        y_true[y_true >= 0.44] = True; y_true[y_true < 0.44] = False
        y_pred[y_pred >= 0.44] = True; y_pred[y_pred < 0.44] = False
        return matthews_corrcoef(y_true, y_pred)
    
    def accracy_reg(y_true, y_pred):
        y_true[y_true >= 0.44] = True; y_true[y_true < 0.44] = False
        y_pred[y_pred >= 0.44] = True; y_pred[y_pred < 0.44] = False
        return accuracy_score(y_true, y_pred)
    
    def precision_reg(y_true,y_pred):
        y_true[y_true >= 0.44] = True; y_true[y_true < 0.44] = False
        y_pred[y_pred >= 0.44] = True; y_pred[y_pred < 0.44] = False
        return average_precision_score(y_true, y_pred)

    #use the best results for prediction
    reg = MLPRegressor(hidden_layer_sizes=(31, ),activation='relu', learning_rate='invscaling',learning_rate_init=0.003)

    method_original = {'accuracy':make_scorer(accracy_reg),'precision':make_scorer(precision_reg),'AUC':'roc_auc','MCC':make_scorer(mc_reg),'r2':'r2'}

    results_original = cross_validate(reg, X, y, cv=kf, scoring=method_original, return_train_score=False)
    
    #out put the scores
    score_original = pd.DataFrame([[np.mean(results_original['test_accuracy']),np.mean(results_original['test_precision']),
                                    np.mean(results_original['test_AUC']),np.mean(results_original['test_MCC']),
                                    np.mean(results_original['test_r2'])]],
                                    columns=['accuracy','precision','AUC','MCC','R2'])
    
    #output the predictions
    # create an empty dictionary for output
    predictions = {}
    index = 0
    for train, test in kf.split(X):
        X_train, X_test, y_train, y_test = X[train], X[test], y[train], y[test]
        reg.fit(X_train,y_train)
        y_predict = reg.predict(X_test)
        predictions[index] = pd.DataFrame(np.column_stack((predicted_original,y_original)),columns=['Predicted','Original'])
        index+=1
        
    return score_original,predictions

In [96]:
#call the evaluation function
score_original,predictions = Neural_Network_Prediciton(df_Original)

#print score
score_original

,accuracy,precision,AUC,MCC,R2
0,0.728692,0.662371,0.811459,0.455022,0.235667


In [97]:
#input 0-9 to retrieve the different folds
predictions[9]


,Predicted,Original
0,0.412097,0.23
1,0.329471,0.26
2,0.426851,0.05
3,0.491620,1.00
4,0.307906,0.54
5,0.335298,0.05
6,0.417080,0.08
7,0.462821,0.23
8,0.577779,0.26
9,0.374297,0.55


In [28]:
result_original = Neural_Network_Prediciton(df_WithGan)
result_original

,accuracy,precision,AUC,MCC,R2
0,0.621298,0.576914,0.670701,0.244327,0.090771
